# MNIST
## inspired from https://github.com/gato/tensor-on-pi/blob/master/Convolutional%20Neural%20Network%20digit%20predictor.ipynb


In [4]:
import os
import shutil
import time

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Dense, Flatten, Softmax, Conv2D, Dropout, MaxPooling2D

print(tf.__version__)


2.4.0-rc2


In [ ]:
mnist = tf.keras.datasets.mnist.load_data()
(x_train, y_train), (x_test, y_test) = mnist

In [ ]:
HEIGHT, WIDTH = x_train[0].shape
NCLASSES = tf.size(tf.unique(y_train).y)
print("Image height x width is", HEIGHT, "x", WIDTH)
tf.print("There are", NCLASSES, "classes")

In [ ]:
def get_model():
 
    model = Sequential([
            Conv2D(64, kernel_size=3,
                   activation='relu', input_shape=(WIDTH, HEIGHT, 1)),
            MaxPooling2D(2),
            Conv2D(32, kernel_size=3,
                   activation='relu'),
            MaxPooling2D(2),
            Flatten(),
            Dense(400, activation='relu'),
            Dense(100, activation='relu'),
            Dropout(.25),
            Dense(10),
            Softmax()
        ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
BUFFER_SIZE = 5000
BATCH_SIZE = 100

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.expand_dims(image, -1)
    return image, label


def load_dataset(training=True):
    """Loads MNIST dataset into a tf.data.Dataset"""
    (x_train, y_train), (x_test, y_test) = mnist
    x = x_train if training else x_test
    y = y_train if training else y_test
    # One-hot encode the classes
    y = tf.keras.utils.to_categorical(y, NCLASSES)
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(scale).batch(BATCH_SIZE)
    if training:
        dataset = dataset.shuffle(BUFFER_SIZE).repeat()
    return dataset

In [ ]:
NUM_EPOCHS = 10
STEPS_PER_EPOCH = 100

model = get_model()
train_data = load_dataset()
validation_data = load_dataset(training=False)

OUTDIR = "mnist_digits/"
checkpoint_callback = ModelCheckpoint(
    OUTDIR, save_weights_only=True, verbose=1)
tensorboard_callback = TensorBoard(log_dir=OUTDIR)
t1 = time.perf_counter()
history = model.fit(
    train_data, 
    validation_data=validation_data,
    epochs=NUM_EPOCHS, 
    steps_per_epoch=STEPS_PER_EPOCH,
    verbose=2,
    callbacks=[checkpoint_callback, tensorboard_callback]
)
t2 = time.perf_counter()
print("training took: {:4.4f} secs.".format(t2 - t1))